# Title: Regenerative Cooling Rocket Engine

Date: 09 Dec 2024

Group member 1:
- Name: Shah, Sharvi
- University ID: 3036932704
- Email: sharvishah@berkeley.edu

Group member 2:
- Name: Parker, Nicole
- University ID: 3036600958
- Email: nicoleaparker@berkeley.edu

-------

## Problem Statement (200 words maximum)
In liquid-fuel rocket engines, a critical design consideration is the temperature of the combustion chamber walls, as the heat generated by fuel combustion can melt through the chamber walls if they are thermally unregulated. A common solution is regenerative cooling, where some fuel is pumped into the combustion chamber through holes in the chamber walls called cooling channels, absorbing heat from the combustion chamber walls en route to the combustion chamber. This simultaneously regulates the chamber wall temperature and makes the combustion process more efficient.
Design a regenerative cooling circuit for an idealized liquid-fuel rocket engine, such that appropriate thrust can be generated in the combustion chamber while the steady-state wall temperature is below the melting temperature of your material of choice. Assume that the working fluid is ethanol, and that fuel must enter the combustion chamber in the liquid phase.

-------

## Background information and Summary (250 words maximum)
Rocket engines, especially those using liquid fuel, generate high temperatures due to the combustion of fuel and oxidizer. In a typical liquid-fueled rocket engine, the temperatures in the combustion chamber can exceed 3,000 Kelvin. Without proper cooling, the engine parts could completely melt causing an explosion and/or rocket faliure. This could put astronauts and others on the aircraft in fatal danger, so it is extremely important to make sure the rocket engines are able to withstand extremely high temperatures.

One of the most important applications of liquid cooling is in the walls of the combustion chamber and the nozzle throat. These areas experience the highest thermal stress because they are directly exposed to the hot gases produced by the fuel-oxidizer reaction. The engine walls are often lined with a network of coolant passages, to avoid any structural failure. As the liquid fuel moves around the coolant passages, it absorbs heat from its surroundings and takes it away from the combustion chamber. This prevents the walls of the combustion chamber from reaching dangerously high temperatures.

Liquid cooling allows rocket engines to operate at higher performance levels, pushing the limits of temperature and pressure without putting any of the hardware at structural risk. By managing heat super efficiently, engineers can design engines that produce more thrust and reduce the risk of overheating by a large amount. This is particularly important for achieving higher specific impulse (a measure of engine efficiency) and overall engine performance.

Regulating the temperature in the chamber walls and improving the efficiency of the combustion process will ensure the structural safety of the rocket. Even more importantly, it will make sure those on board the rocket will be in a safe environment. The reliability and performance of the rocket will be majorly improved, and improving efficiency will also reduce overall costs for projects such as these.

-------

## Modeling the Thermodynamic Process



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
!pip3 install sympy
import sympy as sp
!pip3 install CoolProp
from CoolProp.CoolProp import PropsSI

## Equations & Given Variables
If the combustion engine walls are made of stainless steel, a common material used in rocket engines, we know the melting point is 1773K. Additionally, the coolant ethanol has the boiling point of 351K, and starts at the temperature of 300K. We are hoping to manage the heat transfer and keep the wall temperature below melting point while also not allowing the ethanol to reach boiling point. We can assume a heat load of 1000 kW needs to be removed from the combustion chamber.


We can use energy balance equations that we learned in class to understand this problem. Assume all temperatures are in Kelvin:

### Heat Transfer in Single-Pass System:
$$
Q_{\text{total}} = \dot{m} \times c_p \times (T_{\text{out}} - T_{\text{in}})
$$

$Q_{\text{total}}$ is the total heat to be removed [W].
$\dot{m}$ is the mass flow rate of ethanol [kg/s].
$c_p$ is the specific heat capacity of ethanol [2460 J/kg·K].
$T_{\text{out}}$ is the outlet temperature.
$T_{\text{in}}$ is the inlet temperature [300 K].

### Heat Transfer in Double-Pass System:
$$
Q_{\text{per pass}} = \frac{Q_{\text{total}}}{2} = \dot{m} \times c_p \times (T_{\text{1}} - T_{\text{in}})
$$
$$
Q_{\text{per pass}} = \dot{m} \times c_p \times (T_{\text{out}} - T_{\text{1}})
$$

$Q_{\text{per pass}}$ is the heat removed in each pass [W].
$T_{\text{1}}$ is the intermediate temperature between passes.

### System Constraints
$$
T_{\text{out}} \leq T_{\text{boiling}} = 351\text{ K}
$$
$$
T_{\text{wall}} \leq T_{\text{melting}} = 1773\text{ K}
$$

### Exergy Destruction
$$
X_d = Q_{\text{total}} \times (1 - \frac{T_{\text{0}}}{T_{\text{wall}}})
$$

$T_{\text{0}}$ is the initial room temperature.

Looking at switching between single vs double pass cooling processes can keep the same fundamental cooling system yet provide better temperature control by changing how the coolant flows through the chamber. A single-pass system means the ethanol will absorb all 1000 kW of heat in one continuous process, and create steep temperature gradients and thermal stress along the chamber walls. The double-pass system splits this heat absorption into two 500 kW stages. This allows for a more gradual temperature change in the cooling process. This modification could improve the cooling system by reducing thermal stress and providing better temperature control.

In [41]:
# Material Properties
T_melting = 1773    # Stainless steel melting point
T_boiling = 351     # Ethanol boiling point
cp = 2460           # Ethanol specific heat capacity [J/kg·K]

# Initial/Boundary Conditions
T_0 = 300     # Room temperature
T_in = 300         # Ethanol inlet temperature [K]
Q_total = 1000000  # Heat to remove [W] 

#1000kW is an assumption for the heat load for the combustion chamber in a liquid-fuel rocket engine.

## Single Pass System

In [42]:
dT_max = T_boiling - T_0
m_dot = Q_total / (cp * dT_max)

m_dot

7.970667941973537

In the single-pass cooling system, ethanol enters the cooling channels at 300K and absorbs 1000 kW of heat from the combustion chamber walls in one continuous process. This causes the ethanol temperature to rise by 51K (from 300K to 351K), requiring a mass flow rate of 7.97 kg/s to safely remove the heat while keeping the coolant below its boiling point. While this approach is simpler to implement, it creates a steep temperature gradient along the chamber walls.

## Double Pass System

In [43]:
dT_per_pass = dT_max / 2
T_1 = T_0 + dT_per_pass
T_2 = T_1 + dT_per_pass
Q_per_pass = Q_total / 2

Q_per_pass

500000.0

In the double-pass system, we use the same total mass flow rate but split the cooling into two stages, with each stage handling 500 kW of heat removal. The ethanol first rises by 25.5K (from 300K to 325.5K) in the first pass, then another 25.5K (reaching 351K) in the second pass. This more gradual temperature rise reduces thermal stress on the chamber walls and creates more uniform cooling. Though slightly more complex to manufacture, this approach provides better temperature control and reduces the risk of localized hot spots.

## Exergy Calculations

In [48]:
X_d_single = Q_total * (1 - T_0/T_melting)

X_d_single

830795.2622673435

In [49]:
X_d_first_pass = Q_per_pass * (1 - T_0/T_1)
X_d_second_pass = Q_per_pass * (1 - T_1/T_2)
X_d_double = X_d_first_pass + X_d_second_pass

X_d_double

75495.2932372287

In our exergy analysis, we calculate how much useful energy is lost during the cooling process due to temperature differences between the coolant and the chamber walls. The single-pass system has a much larger loss as ethanol absorbs 1000 kW of heat in a single temperature jump from 300K to 351K. The double-pass system splits this into two smaller jumps of 500 kW each, reducing the total energy quality loss because smaller temperature differences at each stage create fewer irreversibilities in the process.

## Determining the COP

### For single pass:
$$
COP_{\text{single}} = \frac{Q_{\text{total}}}{W_{\text{input}}} = \frac{Q_{\text{total}}}{Q_{\text{hot}} - Q_{\text{cold}}}
$$

### For double pass (considering each stage):
$$
COP_{\text{double}} = \frac{Q_{\text{pass1}}}{W_{\text{input1}}} + \frac{Q_{\text{pass2}}}{W_{\text{input2}}}
$$

In [51]:
COP_single = Q_total / (Q_total * (T_boiling - T_0)/T_0)

COP_single

5.882352941176471

In [52]:
Q_pass1 = Q_total/2
W_input1 = Q_pass1 * (T_1 - T_0)/T_0

Q_pass2 = Q_total/2
W_input2 = Q_pass2 * (T_2 - T_1)/T_1

COP_double = (Q_pass1 + Q_pass2)/(W_input1 + W_input2)

COP_double

12.244322189307379

The double-pass system is approximately twice as efficient at removing heat for the same work input. The higher COP in the double-pass system occurs because we're using smaller temperature differences at each stage, which requires less work to move the same amount of heat. This means we are getting more cooling effect per unit of work input.

----

## Conclusions (150 words max)

Here are our conclusions...

A double-pass cooling system would be a good way to improve the cooling in the rocket combustion engine in comparison to a single-pass system. Splitting the total heat removal of an estimated 1000 kW into two 500 kW stages creates more uniform cooling in the engine and reduces thermal stress on the chamber walls. Our calculations show that the double-pass system has a significantly higher Coefficient of Performance (COP of 12.24 versus 5.88), indicating it removes heat more efficiently for the same work input. Additionally, our exergy analysis demonstrates that using smaller temperature differences in each pass reduces irreversibilities in the system. A double-pass cooling system, though, is more expensive than a single-pass one. There is more complexity in a double-pass cooling system, and more components are required to support such a process. However, since a double-pass system can offer better heat exchange efficiency and cooling performance, we beleive the higher cost is justifyable.

The difference in exergy and COP values between single- and double-pass cooling systems is quite large, indicating these calculations may be way too simple to be explaining such a complex process. It is important to note this and proceed with caution and knowledge of this oversimplification. The values may be a bit off, but we can assume there will be a clear improvement in efficiency between the single- and double-pass cooling systems.